## 标题:python中类的构造方法`__new__`有什么用,和`__init__`有什么区别

### 1. 常规写法
1. 在常规写法中，我们就是使用`__init__`来初始化一个类的实例。

In [55]:
class A(object):
    def __init__(self, *args,**kwargs):
        self.args = args
        self.kwargs = kwargs
        print(f"init a class, {self.args=}, {self.kwargs}")


a = A(1,2,3,gzh='统计学人')
b = A(1,2,3,date = "2022-12-25")


init a class, self.args=(1, 2, 3), {'gzh': '统计学人'}
init a class, self.args=(1, 2, 3), {'date': '2022-12-25'}


In [56]:
print(a)
print(b)

### 2. 添加`__new__`属性

In [57]:
from datetime import datetime

class A(object):

    def __new__(cls, *args, **kwargs) :
        # print("*"*80)
        print(f"__new__ is called in {datetime.now()}")
        # print("*"*80)

        return super().__new__(cls)

    def __init__(self, *args,**kwargs):
        self.args = args
        self.kwargs = kwargs
        print(f"__init__ is called in {datetime.now()}")
        # print(f"{self.args=}, {self.kwargs}")


a = A(1,2,3,gzh='统计学人')
# b = A(1,2,3,date = "2022-12-25")

__new__ is called in 2022-12-25 13:28:36.702439
__init__ is called in 2022-12-25 13:28:36.702623


查看上面的运行时间，可以看出来：先运行`__new__`，再运行`__init__`，这里也不得不介绍一下这个到底是怎么回事
### `__new__`和`__init__`的关系
1. `__new__` 负责`对象创建`。
2.  `__init__` 负责`对象初始化`。

#### `tips:`
1. 说白了，以前我们只是知道一个对象的需要初始化，但是不知道这个对象在初始化之前需要创建一个对象。这一次就是更加仔细的把一个对象的创建过程，表现出来。
2. 这么做图什么？图的是对每一个步骤、每一个数据对象的精准控制。

### 使用注意事项
1. `__new__`的第一个参数是`cls`，而不是`self`，这个是不同于`__init__`的。
2. `__new__`一般来说，最好是需要`return`的, 但是`__init__`不需要返回值。
3. `__new__`通常情况下，最好不要用。

## 使用`__new__`的场景
### 1. 不可改变类型的子类
有这样一个场景，需要使用一个新的数据类型：大写类型的tuple，即不管输入什么字符串列表，都需要返回的是大写字符串的tuple样式：

example: `myclass(['a','hello', 'world']) => ('A', 'HELLO', 'WORDLD')`


Q： 有的人说，这还不简单，就是创建一个返回为`tuple`的类呗，为什么要继承这个`tuple`类？

A： 这么做肯定是可以的，但是从性能角度考虑，`tuple`类型是全部都是使用C语言编写的，直接使用`tuple`的内置方法来实现，效率更高。你写的任何python代码，都是没有内置C更快的。

In [58]:
class UppercaseTuple(tuple):
    def __init__(self, iterable):
        print(f"init {iterable}")
        for i , arg in enumerate(iterable):
            self[i] = arg.upper()


def inheriting_immutable_uppercase_tuple_example():
    print("UPPERCASE TUPLE EXAMPLE")
    print(UppercaseTuple(["hello", "world"]))

inheriting_immutable_uppercase_tuple_example()

UPPERCASE TUPLE EXAMPLE
init ['hello', 'world']


TypeError: 'UppercaseTuple' object does not support item assignment

#### 代码报错
上面代码是使用`__init__`写了一个类，希望实现我们的需求。但是没完成？，原因如下：
1. 因为tuple是一个不可变类型的数据格式。
2. tuple数据类型在被创建之后，就不能在使用`__init__`来进行初始化了。
3. 因此我们要赶在tuple被创建的时候，就要开始动手做一些事情了。



In [ ]:
class UppercaseTuple(tuple):
    def __new__(cls, iterable):
        upper_iterable = (s.upper() for s in iterable)
        return super().__new__(cls, upper_iterable)
    # def __init__(self, iterable):
    #     print(f"init {iterable}")
    #     for i , arg in enumerate(iterable):
    #         self[i] = arg.upper()


def inheriting_immutable_uppercase_tuple_example():
    print("UPPERCASE TUPLE EXAMPLE")
    print(UppercaseTuple(["hello", "world"]))

inheriting_immutable_uppercase_tuple_example()

UPPERCASE TUPLE EXAMPLE
('HELLO', 'WORLD')


### 单例模式
单例模式最初的定义出现于《设计模式》：“保证一个类仅有一个实例，并提供一个访问它的全局访问点。”
单例的使用主要是在需要保证全局只有一个实例可以被访问的情况，比如系统日志的输出、操作系统的任务管理器等。

说的更加直白一点：不管是macos还是window系统，电脑上都是有一个垃圾桶🚮。而且只有一个。你不会在电脑上放两个垃圾桶吧。那么强制只有一个垃圾桶，而且可以在任何地方把文件丢到垃圾桶里面，这种设计模式就是`单例模式`

In [ ]:
class Singleton:
    """单例模式的代码""" 
    _instance = None 

    def __new__(cls, *args, **kwargs):
        if cls._instance is None:
            cls._instance = super().__new__(cls, *args, **kwargs)
        
        return cls._instance

def singleton_example():
    print("SINGLETON EXAMPLE")
    x = Singleton()
    y = Singleton()

    print(f"{x is y=}")

singleton_example()

SINGLETON EXAMPLE
x is y=True


#### 说明
1. 上面就是一个简单的单例模式模式的案例，x,y其实关联到一个实例上。

为了更加清楚单例模式模式，那我们来举例子：
1. 你是光明县的县长，叫梅县长。
2. 你们厂里有一台污水处理厂，而且只有一台。
3. 有一天，一个养猪场的厂长来找你，说：“我这里有很多猪，每天拉屎拉尿的，有很多污水，需要处理，你给我盖污水处理厂一个吧”。
4. 又有一天，一个化肥厂的厂长来找你，说：“我这里每天要生产很多化肥，但是有很多污水，你给我盖个污水处理厂吧”。
5. 有很多人来找你，希望你给他们盖污水处理厂。
6. 你作为县长，肯定说没问题。说：“你们养猪场的污水处理厂，我批准了。你们化肥长的污水处理厂我也批准了，，，，，”。
7. 但是你实际上，只是盖了一个污水处理厂。虽然允诺了盖了十几个污水厂。只不过是暗地里修了很多管道，通往一个污水处理厂罢了。
8. 污水处理厂，最重要的还是设备，一台设备几百万。但是管道才多少钱。这样下来，利用政府项目，捞了多少油水。

In [ ]:
class Wushui:
    _loaded = {}

    def __new__(cls, factory_name:str, to_user:str):
        if (factory := cls._loaded.get(factory_name)) is not None:
            print(f"{factory_name} 已经盖过了，就用之前的吧")
            return factory
        
        factory = super().__new__(cls)
        cls._loaded[factory_name] = factory
        factory._init_from_user(factory_name, to_user)
        return factory

    def _init_from_user(self, factory_name:str, to_user:str):
        print(f"给 {to_user} 盖的 {factory_name}")
        self.to_user = to_user

def dangguan():
    
    print("当官期间")
    x = Wushui(factory_name="县南污水处理厂", to_user="养猪")
    y = Wushui(factory_name="县南污水处理厂", to_user="化肥厂")
    print(f"{x is y = }")
    z = Wushui(factory_name="县北污水处理厂", to_user="钢铁厂")

dangguan()


当官期间
给 养猪 盖的 县南污水处理厂
县南污水处理厂 已经盖过了，就用之前的吧
x is y = True
给 钢铁厂 盖的 县北污水处理厂


#### 说明

上面的运行结果可以看出来:
1. 先给`养猪场`盖一个`县南污水处理厂`
2. 后来虽然给`化肥厂`表面上也盖了一个所谓处理厂，但是县长的小算盘：依然使用`县南污水处理厂`
3. 钢铁污水多，还是另外盖一个新的`县北污水处理厂`.

这样看下来，基本上就是对使用`__new__`实现`单例模式`非常清晰了吧。


#### 继续
到这里，你作为老谋深算的梅县长，肯定也是知道并没有结束，上面方案虽然是把问题解决了，但是我们要做的天衣无缝，不然很容易被撸下马。做县长肯定是没那么简单的。

1. 你也知道，不同类型的污水，肯定是不能混合处理的。养猪的污水和生活污水是不能混合的，钢铁厂的污水和化肥厂的污水肯定也是不能混合的。
2. 虽然对外你说的是建立了几个厂，但是你实际上就一个厂处理，万一一个厂发生爆炸了，这可怎么办。
3. 但是出于成本考虑，需要把不同的污水放在一个厂处理。这可怎么办？
4. 那就这么做：一个厂，按照污水的处理需求，分为：`常规污水`,`重金属污水`,`生活污水`,不同污水来了，就什么方式处理。反正就是见招拆招.
5. 而且污水处理厂，升级改造，还是能赚到不少钱的。美滋滋。开干！

In [ ]:
class BigFactory:
    _registry = {}
    _wushui_type = {'养猪场':'常规污水', '东苑':'生活污水', '钢铁厂':'重金属污水', '化肥厂':'常规污水'}

    def __init_subclass__(cls, prefix, **kwargs) -> None:
        super().__init_subclass__(**kwargs)
        cls._registry[prefix] = cls

    
    def __new__(cls, wushui:str):
        send_name,sep, size = wushui.partition("送来了")

        if sep:
            size = size 
        else:
            size = 0
            # prefix = "常规污水"

        prefix = cls._wushui_type.get(send_name, "常规污水")
        
        
        subclass = cls._registry[prefix]
        obj = object.__new__(subclass)
        obj.size = size 
        obj.send_name = send_name
        obj.wushui_type = prefix
        return obj

class smallfactory1(BigFactory, prefix="常规污水"):
    def process(self):
        return '车间1', self.send_name, self.wushui_type, self.size 

class smallfactory2(BigFactory,prefix="生活污水"):
    def process(self):
        return '车间2', self.send_name, self.wushui_type, self.size 

class smallfactory3(BigFactory,prefix="重金属污水"):
    def process(self):
        return '车间3', self.send_name, self.wushui_type, self.size 




def dangguan():
    print(BigFactory(wushui="养猪场送来了10吨").process())
    print(BigFactory(wushui="东苑送来了20吨").process())
    print(BigFactory(wushui="化肥厂送来了30吨").process())
    print(BigFactory(wushui="钢铁厂送来了40吨").process())

dangguan()

('车间1', '养猪场', '常规污水', '10吨')
('车间2', '东苑', '生活污水', '20吨')
('车间1', '化肥厂', '常规污水', '30吨')
('车间3', '钢铁厂', '重金属污水', '40吨')


#### 说明
1. 可以看出来，我们这个新的智能工厂，会自动的处理各种类型的污水，按照污水的来源，分发到不同的车间。
2. 这个工厂不管是表面服务于各个生产工厂，还是说内部的污水处理逻辑。都是相当强大的。
3. 这个钱虽然是贪了不少，但是通过科学的手段来贪，可谓是秒哇！

## 参考链接
1. https://juejin.cn/post/6844904065302790158
2. https://peps.python.org/pep-0487/#requiring-an-explicit-decorator-on-init-subclass
3. https://www.cnblogs.com/zhanhaixiang/p/15615286.html
4. https://www.51cto.com/article/711136.html
5. https://blog.csdn.net/luoweifu/article/details/82732313
6. https://zhuanlan.zhihu.com/p/21379984
7. https://zhuanlan.zhihu.com/p/225834335
8. https://stackoverflow.com/questions/45400284/understanding-init-subclass
9. https://www.youtube.com/watch?v=-zsV0_QrfTw
10. https://www.programiz.com/python-programming/methods/string/partition